In [1]:
import pandas as pd

In [2]:
df_posts = pd.read_parquet("../data/base.parquet").replace({"nan": pd.NA})

In [7]:
users = (
    pd.concat([
        df_posts.dropna(subset=["quotes"]).groupby("user_id").count().quotes,
        df_posts.dropna(subset=["body"]).groupby("user_id").count().body,
        df_posts.drop_duplicates(subset=["user_id", "thread_id"]).groupby("user_id").count().thread_id,
        df_posts.groupby("user_id").date.min().reset_index().rename({"date":"activity_first"},axis=1).set_index("user_id"),
        df_posts.groupby("user_id").date.max().reset_index().rename({"date":"activity_last"},axis=1).set_index("user_id"),
        df_posts.groupby("user_id").community.unique(),
    ], axis=1)
    .reset_index()
    .rename({
        "quotes": "num_quotes",
        "body": "num_posts",
        "thread_id": "num_threads",
        "index": "user_id",
    }, axis=1)
    .fillna(0)
)

In [10]:
users.community = users.community.apply(lambda x: x[0])

In [12]:
users.to_parquet("../data/users.parquet", compression="gzip")